<a href="https://colab.research.google.com/github/ahmedoumar/ArSarcasm/blob/master/Fine_tune_MARBERT_for_ArSarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download MARBERT checkpoint

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!wget https://huggingface.co/UBC-NLP/MARBERT/resolve/main/MARBERT_pytorch_verison.tar.gz

--2021-04-13 10:49:27--  https://huggingface.co/UBC-NLP/MARBERT/resolve/main/MARBERT_pytorch_verison.tar.gz
Resolving huggingface.co (huggingface.co)... 34.201.172.85
Connecting to huggingface.co (huggingface.co)|34.201.172.85|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/UBC-NLP/MARBERT/85bfec76f38cba4bc2e6cd02a959016de37ba93de4c850a7d175811dce4e8adc [following]
--2021-04-13 10:49:27--  https://cdn-lfs.huggingface.co/UBC-NLP/MARBERT/85bfec76f38cba4bc2e6cd02a959016de37ba93de4c850a7d175811dce4e8adc
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 52.84.162.46, 52.84.162.9, 52.84.162.75, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|52.84.162.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607066087 (579M) [application/x-gzip]
Saving to: ‘MARBERT_pytorch_verison.tar.gz’

MARBERT_pytorch_ver 100%[===================>] 578.94M   218MB/s    in 2.7s    

2021-04-13 10:

In [ ]:
!tar -xvf MARBERT_pytorch_verison.tar.gz

MARBERT_pytorch_verison/
MARBERT_pytorch_verison/pytorch_model.bin
MARBERT_pytorch_verison/config.json
MARBERT_pytorch_verison/vocab.txt


In [ ]:
!wget https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_train.tsv
!wget https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_test.tsv

--2021-04-13 10:49:39--  https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149495 (146K) [text/plain]
Saving to: ‘UBC_AJGT_final_shuffled_train.tsv’

UBC_AJGT_final_shuf 100%[===================>] 145.99K  --.-KB/s    in 0.02s   

2021-04-13 10:49:40 (9.32 MB/s) - ‘UBC_AJGT_final_shuffled_train.tsv’ saved [149495/149495]

--2021-04-13 10:49:40--  https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
H

In [ ]:
!mkdir -p AJGT
!mv UBC_AJGT_final_shuffled_train.tsv ./AJGT/UBC_AJGT_final_shuffled_train.tsv
!mv UBC_AJGT_final_shuffled_test.tsv ./AJGT/UBC_AJGT_final_shuffled_test.tsv

In [ ]:
!pip install GPUtil pytorch_pretrained_bert transformers

     |████████████████████████████████| 133kB 6.5MB/s 
     |████████████████████████████████| 2.2MB 10.5MB/s 
     |████████████████████████████████| 102kB 11.7MB/s 
     |████████████████████████████████| 3.3MB 34.4MB/s 
     |████████████████████████████████| 870kB 43.8MB/s 
     |████████████████████████████████| 7.4MB 53.7MB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=202d6c7372f8a322cedd74b1ba321adf0f0ac38f36747b9b7cb6505dc4c2584b
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
  Created wheel for boto3: filename=boto3-1.17.50-py2.py3-none-any.whl size=128779 sha256=0d9fb3d5e7159fea611c3df15b259763b30b03dbe816722beea07cd3d167eb50
  Stored in directory: /root/.cache/pip/wheels/28/e5/43/ef6fc36c3008477a35f9324c0e490c7aa20f7b51993a388267
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sh

In [ ]:
!git clone https://github.com/iabufarha/ArSarcasm.git

Cloning into 'ArSarcasm'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 85 (delta 37), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (85/85), done.


# Fine-tuning code

In [ ]:
!pip3 install sentencepiece

     |████████████████████████████████| 1.2MB 9.1MB/s 


In [ ]:
# (1)load libraries 
import json, sys, regex
import torch
import GPUtil
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
##----------------------------------------------------
from transformers import *
from transformers import XLMRobertaConfig
from transformers import XLMRobertaModel
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, XLMRobertaModel
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("your device ", device)

your device  cuda


In [ ]:

def create_label2ind_file(file, label_col):
	labels_json={}
	#load train_dev_test file
	df = pd.read_csv(file, sep=",")
	df.head(5)
	#get labels and sort it A-Z
	labels = df[label_col].unique()
	labels.sort()
	#convert labels to indexes
	for idx in range(0, len(labels)):
		labels_json[labels[idx]]=idx
	#save labels with indexes to file
	with open(label2idx_file, 'w') as json_file:
		json.dump(labels_json, json_file)


In [ ]:

def data_prepare_BERT(file_path, lab2ind, tokenizer, content_col, label_col, MAX_LEN):
	# Use pandas to load dataset
	df = pd.read_csv(file_path, delimiter='\t', header=0); df[['sarcasm', 'tweet']] = df['sarcasm,tweet'].str.split(',', 1, expand=True)
	df = df[df[content_col].notnull()]
	df = df[df[label_col].notnull()]
	print("Data size ", df.shape)
	# Create sentence and label lists
	sentences = df[content_col].values
	sentences = ["[CLS] " + sentence+ " [SEP]" for sentence in sentences]
	print ("The first sentence:")
	print (sentences[0])
	# Create sentence and label lists
	labels = df[label_col].values
	#print (labels)
	labels = [lab2ind[i] for i in labels]
	# Import the BERT tokenizer, used to convert our text into tokens that correspond to BERT's vocabulary.
	tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
	print ("Tokenize the first sentence:")
	print (tokenized_texts[0])
	#print("Label is ", labels[0])
	# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
	input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
	print ("Index numbers of the first sentence:")
	print (input_ids[0])
	# Pad our input seqeunce to the fixed length (i.e., max_len) with index of [PAD] token
	# ~ input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
	pad_ind = tokenizer.convert_tokens_to_ids(['[PAD]'])[0]
	input_ids = pad_sequences(input_ids, maxlen=MAX_LEN+2, dtype="long", truncating="post", padding="post", value=pad_ind)
	print ("Index numbers of the first sentence after padding:\n",input_ids[0])
	# Create attention masks
	attention_masks = []
	# Create a mask of 1s for each token followed by 0s for padding
	for seq in input_ids:
		seq_mask = [float(i > 0) for i in seq]
		attention_masks.append(seq_mask)
	# Convert all of our data into torch tensors, the required datatype for our model
	inputs = torch.tensor(input_ids)
	labels = torch.tensor(labels)
	masks = torch.tensor(attention_masks)
	return inputs, labels, masks


In [ ]:
# Function to calculate the accuracy of our predictions vs labels
# def flat_accuracy(preds, labels):
#	  pred_flat = np.argmax(preds, axis=1).flatten()
#	  labels_flat = labels.flatten()
#	  return np.sum(pred_flat == labels_flat) / len(labels_flat)
def flat_pred(preds, labels):
	pred_flat = np.argmax(preds, axis=1).flatten()
	labels_flat = labels.flatten()
	return pred_flat.tolist(), labels_flat.tolist()

In [ ]:

def train(model, iterator, optimizer, scheduler, criterion):
	
	model.train()
	epoch_loss = 0
	for i, batch in enumerate(iterator):
		# Add batch to GPU
		batch = tuple(t.to(device) for t in batch)
		# Unpack the inputs from our dataloader
		input_ids, input_mask, labels = batch
		outputs = model(input_ids, input_mask, labels=labels)
		loss, logits = outputs[:2]
		# delete used variables to free GPU memory
		del batch, input_ids, input_mask, labels
		optimizer.zero_grad()
		if torch.cuda.device_count() == 1:
			loss.backward()
			epoch_loss += loss.cpu().item()
		else:
			loss.sum().backward()
			epoch_loss += loss.sum().cpu().item()
		optimizer.step()
		torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Gradient clipping is not in AdamW anymore
		# optimizer.step()
		scheduler.step()
	# free GPU memory
	if device == 'cuda':
		torch.cuda.empty_cache()
	return epoch_loss / len(iterator)

In [ ]:

def evaluate(model, iterator, criterion):
	model.eval()
	epoch_loss = 0
	all_pred=[]
	all_label = []
	with torch.no_grad():
		for i, batch in enumerate(iterator):
			# Add batch to GPU
			batch = tuple(t.to(device) for t in batch)
			# Unpack the inputs from our dataloader
			input_ids, input_mask, labels = batch
			outputs = model(input_ids, input_mask, labels=labels)
			loss, logits = outputs[:2]
			# delete used variables to free GPU memory
			del batch, input_ids, input_mask
			if torch.cuda.device_count() == 1:
				epoch_loss += loss.cpu().item()
			else:
				epoch_loss += loss.sum().cpu().item()
			# identify the predicted class for each example in the batch
			probabilities, predicted = torch.max(logits.cpu().data, 1)
			# put all the true labels and predictions to two lists
			all_pred.extend(predicted)
			all_label.extend(labels.cpu())
	accuracy = accuracy_score(all_label, all_pred)
	f1score = f1_score(all_label, all_pred, average='macro') 
	recall = recall_score(all_label, all_pred, average='macro')
	precision = precision_score(all_label, all_pred, average='macro')
	report = classification_report(all_label, all_pred)
	return (epoch_loss / len(iterator)), accuracy, f1score, recall, precision



In [ ]:

def fine_tuning(config):
	#---------------------------------------
	print ("[INFO] step (1) load train_test config file")
	# config_file = open(config_file, 'r', encoding="utf8")
	# config = json.load(config_file)
	task_name = config["task_name"]
	content_col = config["content_col"]
	label_col = config["label_col"]
	train_file = config["data_dir"]+config["train_file"]
	dev_file = config["data_dir"]+config["dev_file"]
	sortby = config["sortby"]
	max_seq_length= int(config["max_seq_length"])
	batch_size = int(config["batch_size"])
	lr_var = float(config["lr"])
	model_path = config['pretrained_model_path']
	num_epochs = config['epochs'] # Number of training epochs (authors recommend between 2 and 4)
	global label2idx_file
	label2idx_file = config["data_dir"]+config["task_name"]+"_labels-dict.json"
	#-------------------------------------------------------
	print ("[INFO] step (2) convert labels2index")
	create_label2ind_file(train_file, label_col)
	print (label2idx_file)
	#---------------------------------------------------------
	print ("[INFO] step (3) check checkpoit directory and report file")
	ckpt_dir = config["data_dir"]+task_name+"_bert_ckpt/"
	report = ckpt_dir+task_name+"_report.tsv"
	sorted_report = ckpt_dir+task_name+"_report_sorted.tsv"
	if not os.path.exists(ckpt_dir):
		os.mkdir(ckpt_dir)
	#-------------------------------------------------------
	print ("[INFO] step (4) load label to number dictionary")
	lab2ind = json.load(open(label2idx_file))
	print ("[INFO] train_file", train_file)
	print ("[INFO] dev_file", dev_file)
	print ("[INFO] num_epochs", num_epochs)
	print ("[INFO] model_path", model_path)
	print ("max_seq_length", max_seq_length, "batch_size", batch_size)
	#-------------------------------------------------------
	print ("[INFO] step (5) Use defined funtion to extract tokanize data")
	# tokenizer from pre-trained BERT model
	print ("loading BERT setting")
	tokenizer = BertTokenizer.from_pretrained(model_path)
	train_inputs, train_labels, train_masks = data_prepare_BERT(train_file, lab2ind, tokenizer,content_col, label_col, max_seq_length)
	validation_inputs, validation_labels, validation_masks = data_prepare_BERT(dev_file, lab2ind, tokenizer, content_col, label_col,max_seq_length)
	# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
	model = BertForSequenceClassification.from_pretrained(model_path, num_labels=len(lab2ind))
	#--------------------------------------
	print ("[INFO] step (6) Create an iterator of data with torch DataLoader.")
#		  This helps save on memory during training because, unlike a for loop,\
#		  with an iterator the entire dataset does not need to be loaded into memory")
	train_data = TensorDataset(train_inputs, train_masks, train_labels)
	train_dataloader = DataLoader(train_data, batch_size=batch_size)
	#---------------------------
	validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
	validation_dataloader = DataLoader(validation_data, batch_size=batch_size)
	#------------------------------------------
	print ("[INFO] step (7) run with parallel GPUs")
	if torch.cuda.is_available():
		if torch.cuda.device_count() == 1:
			print("Run", "with one GPU")
			model = model.to(device)
		else:
			n_gpu = torch.cuda.device_count()
			print("Run", "with", n_gpu, "GPUs with max 4 GPUs")
			device_ids = GPUtil.getAvailable(limit = 4)
			torch.backends.cudnn.benchmark = True
			model = model.to(device)
			model = nn.DataParallel(model, device_ids=device_ids)
	else:
		print("Run", "with CPU")
		model = model
	#---------------------------------------------------
	print ("[INFO] step (8) set Parameters, schedules, and loss function")
	global max_grad_norm
	max_grad_norm = 1.0
	warmup_proportion = 0.1
	num_training_steps	= len(train_dataloader) * num_epochs
	num_warmup_steps = num_training_steps * warmup_proportion
	### In Transformers, optimizer and schedules are instantiated like this:
	# Note: AdamW is a class from the huggingface library
	# the 'W' stands for 'Weight Decay"
	optimizer = AdamW(model.parameters(), lr=lr_var, correct_bias=False)
	# schedules
	scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler
	# We use nn.CrossEntropyLoss() as our loss function. 
	criterion = nn.CrossEntropyLoss()
	#---------------------------------------------------
	print ("[INFO] step (9) start fine_tuning")
	for epoch in trange(num_epochs, desc="Epoch"):
        
		train_loss = train(model, train_dataloader, optimizer, scheduler, criterion)	  
		print("hello again"); val_loss, val_acc, val_f1, val_recall, val_precision = evaluate(model, validation_dataloader, criterion)
# 		print (train_loss, val_acc)
		# Create checkpoint at end of each epoch
		if not os.path.exists(ckpt_dir + 'model_' + str(int(epoch + 1)) + '/'): os.mkdir(ckpt_dir + 'model_' + str(int(epoch + 1)) + '/')
		model.save_pretrained(ckpt_dir+ 'model_' + str(int(epoch + 1)) + '/')
		epoch_eval_results = {"epoch_num":int(epoch + 1),"train_loss":train_loss,
					  "val_acc":val_acc, "val_recall":val_recall, "val_precision":val_precision, "val_f1":val_f1,"lr":lr_var }
		with open(report,"a") as fOut:
			fOut.write(json.dumps(epoch_eval_results)+"\n")
			fOut.flush()
		#------------------------------------
		report_df = pd.read_json(report, orient='records', lines=True)
		report_df.sort_values(by=[sortby],ascending=False, inplace=True)
		report_df.to_csv(sorted_report,sep="\t",index=False)
	return report_df

# Run fine-tuning for 5 epochs

In [ ]:
train = pd.read_csv('ArSarcasm/dataset/ArSarcasm_train.csv', index_col=False)
test = pd.read_csv('ArSarcasm/dataset/ArSarcasm_test.csv', index_col=False)

In [ ]:
train['sarcasm'].replace({True:"Positive", False:"Negative"}, inplace=True)
test['sarcasm'].replace({True:"Positive", False:"Negative"}, inplace=True)

In [ ]:
del train['dialect']
del train['sentiment']
del train['original_sentiment']
del train['source']

In [ ]:
del test['dialect']
del test['sentiment']
del test['original_sentiment']
del test['source']

In [ ]:
test.head()

,sarcasm,tweet
0,Positive,"""@AbuEmad74241481 @Cesars2014 هههههه حزب الله ..."
1,Negative,"""RT @JannetForster: البنات اللي م صامو بقولكم ..."
2,Positive,اشارة رابعة اشبه بنار تحرق الانقلابيين
3,Negative,"""@EGYPTAIR ماهي مميزات درجه بزنس علماً اني في ..."
4,Negative,ما لا تراه على التلفاز منافسة شديدة بين المرشح...


In [ ]:
train.to_csv(r'train.csv', index=False)
test.to_csv(r'test.csv', index=False)

In [ ]:
test.head()

,sarcasm,tweet
0,Positive,"""@AbuEmad74241481 @Cesars2014 هههههه حزب الله ..."
1,Negative,"""RT @JannetForster: البنات اللي م صامو بقولكم ..."
2,Positive,اشارة رابعة اشبه بنار تحرق الانقلابيين
3,Negative,"""@EGYPTAIR ماهي مميزات درجه بزنس علماً اني في ..."
4,Negative,ما لا تراه على التلفاز منافسة شديدة بين المرشح...


In [ ]:

config={"task_name": "AJGT_MARBERT", #output directory name
             "data_dir": "./", #data directory
             "train_file": "train.csv", #train file path
             "dev_file": "test.csv", #dev file path or test file path
             "pretrained_model_path": 'MARBERT_pytorch_verison', #MARBERT checkpoint path
             "epochs": 10, #number of epochs
             "content_col": "tweet", #text column
             "label_col": "sarcasm", #label column
             "lr": 2e-04, #learning rate
              "max_seq_length": 128, #max sequance length
              "batch_size": 16, #batch shize
              "sortby":"val_acc"} #sort results based on val_acc or val_f1


In [ ]:
report_df = fine_tuning(config)

In [ ]:
#This report is for the next hyperparameters:
"""
epochs = 5
lr = 2e-06
max_seq_length = 128
batch_size = 16
"""
report_df.head(5)

,epoch_num,train_loss,val_acc,val_recall,val_precision,val_f1,lr
0,1,0.355991,0.872038,0.691481,0.783843,0.722881,0.000002
3,4,0.152038,0.871090,0.766704,0.764246,0.765464,0.000002
4,5,0.135789,0.871090,0.762040,0.764514,0.763266,0.000002
2,3,0.185312,0.870142,0.775465,0.762221,0.768531,0.000002
1,2,0.237678,0.867773,0.762389,0.758195,0.760260,0.000002


In [ ]:
report_df2 = fine_tuning(config)

In [ ]:
#This report is for the next hyperparameters:
"""
epochs = 5
lr = 2e-05
max_seq_length = 128
batch_size = 16
"""
report_df2.head(5)

,epoch_num,train_loss,val_acc,val_recall,val_precision,val_f1,lr
0,1,0.355991,0.872038,0.691481,0.783843,0.722881,0.000002
3,4,0.152038,0.871090,0.766704,0.764246,0.765464,0.000002
4,5,0.135789,0.871090,0.762040,0.764514,0.763266,0.000002
2,3,0.185312,0.870142,0.775465,0.762221,0.768531,0.000002
5,1,0.352652,0.868720,0.675506,0.779764,0.708184,0.000020


In [ ]:
report_df3 = fine_tuning(config)

In [ ]:
#This report is for the next hyperparameters:
"""
epochs = 10
lr = 2e-06
max_seq_length = 128
batch_size = 16
"""
report_df3.head(10)

,epoch_num,train_loss,val_acc,val_recall,val_precision,val_f1,lr
14,5,0.135143,0.873934,0.752079,0.771473,0.761125,0.000002
12,3,0.212011,0.873934,0.742752,0.772983,0.756286,0.000002
11,2,0.265641,0.873460,0.727310,0.775144,0.747257,0.000002
0,1,0.355991,0.872038,0.691481,0.783843,0.722881,0.000002
13,4,0.168793,0.871564,0.768153,0.765073,0.766596,0.000002
3,4,0.152038,0.871090,0.766704,0.764246,0.765464,0.000002
4,5,0.135789,0.871090,0.762040,0.764514,0.763266,0.000002
15,6,0.112253,0.871090,0.763206,0.764441,0.763820,0.000002
2,3,0.185312,0.870142,0.775465,0.762221,0.768531,0.000002
5,1,0.352652,0.868720,0.675506,0.779764,0.708184,0.000020


In [ ]:
report_df4 = fine_tuning(config)

In [ ]:
#This report is for the next hyperparameters:
"""
epochs = 10
lr = 2e-04
max_seq_length = 128
batch_size = 16
"""
report_df4.head(10)

,epoch_num,train_loss,val_acc,val_recall,val_precision,val_f1,lr
14,5,0.135143,0.873934,0.752079,0.771473,0.761125,0.000002
12,3,0.212011,0.873934,0.742752,0.772983,0.756286,0.000002
11,2,0.265641,0.873460,0.727310,0.775144,0.747257,0.000002
0,1,0.355991,0.872038,0.691481,0.783843,0.722881,0.000002
13,4,0.168793,0.871564,0.768153,0.765073,0.766596,0.000002
15,6,0.112253,0.871090,0.763206,0.764441,0.763820,0.000002
4,5,0.135789,0.871090,0.762040,0.764514,0.763266,0.000002
3,4,0.152038,0.871090,0.766704,0.764246,0.765464,0.000002
2,3,0.185312,0.870142,0.775465,0.762221,0.768531,0.000002
18,9,0.067870,0.868720,0.745465,0.760946,0.752770,0.000002
